In [1]:
#!/usr/bin/python3
# -*-coding:utf-8
'''
Created on Fri Dec 1 22:22:35 2017

@author: Ray

It took 5.446 hours
'''
import time
import pandas as pd
import numpy as np
from tqdm import tqdm
import os
import utils # written by author
from glob import glob
from datetime import datetime, timedelta
import multiprocessing as mp
import gc # for automatic releasing memory



In [8]:
##################################################
# Load transaction 
##################################################
#input_col = []
transactions = utils.read_multiple_csv('../../input/preprocessed_data/transactions')
#print ('shape of transactions',transactions.shape)
transactions = transactions.head(n = 500)
##################################################
# Convert string to datetime format
##################################################
transactions['membership_expire_date']  = transactions.membership_expire_date.apply(lambda x: datetime.strptime(x, '%Y-%m-%d'))
transactions['transaction_date']  = transactions.transaction_date.apply(lambda x: datetime.strptime(x, '%Y-%m-%d'))



print ('After Loading ...')


100%|██████████| 4/4 [00:26<00:00,  6.66s/it]


After Loading ...


In [9]:
##################################################
# 善用time(時間差)來區隔交易日期時間點的行為
##################################################

def t_diff(x, time_diff):
    # x : the dataframe belong to a each specific user
    # time_diff: 2,3,4,...
    for i in range(1, time_diff):
        x['t-{}_transaction_date'.format(i)] = x['transaction_date'].shift(i)
        x['t-{}_membership_expire_date'.format(i)] = x['membership_expire_date'].shift(i)
        x['t-{}_payment_method_id'.format(i)] = x['payment_method_id'].shift(i)
        x['t-{}_payment_plan_days'.format(i)] = x['payment_plan_days'].shift(i)
        x['t-{}_plan_list_price'.format(i)] = x['plan_list_price'].shift(i)
        x['t-{}_actual_amount_paid'.format(i)] = x['actual_amount_paid'].shift(i)
        x['t-{}_is_auto_renew'.format(i)] = x['is_auto_renew'].shift(i)
        x['t-{}_is_cancel'.format(i)] = x['is_cancel'].shift(i)
    return x
def make_order_number(x):
    # for near 5
    # 1:代表最靠近現在,數字越大代表是越過去的歷史紀錄
    x['order_number_rev'] = list(reversed([i+1 for i in range(x.shape[0])]))
    return x

##################################################
# Main
##################################################
s = time.time()

transactions = transactions.groupby('msno').apply(t_diff, 2)
transactions = transactions.groupby('msno').apply(make_order_number)

gc.collect()
# write
path = '../input/preprocessed_data/transactions_time_diff'

output_col = ['msno','transaction_date','t-1_transaction_date','t-1_membership_expire_date',
              't-1_payment_method_id','t-1_payment_plan_days',
             't-1_plan_list_price','t-1_actual_amount_paid',
             't-1_is_auto_renew','t-1_is_cancel',
             'order_number_rev'
             ]


tran_time_diff = transactions[output_col] 

In [11]:
tran_time_diff = tran_time_diff.dropna() # drop columns whose oder_number == 1
tran_time_diff

,msno,transaction_date,t-1_transaction_date,t-1_membership_expire_date,t-1_payment_method_id,t-1_payment_plan_days,t-1_plan_list_price,t-1_actual_amount_paid,t-1_is_auto_renew,t-1_is_cancel,order_number_rev
2,+++IZseRRiQS9aaSkH6cMYU6bGDcxUieAi/tH67sC5s=,2016-10-23,2015-11-21,2017-01-04,38.0,410.0,1788.0,1788.0,0.0,0.0,1
4,+++hVY1rZox/33YtvDgmKA2Frg/2qhkz12B9ylCvh8o=,2016-12-15,2016-11-16,2016-12-15,41.0,30.0,99.0,99.0,1.0,0.0,4
5,+++hVY1rZox/33YtvDgmKA2Frg/2qhkz12B9ylCvh8o=,2017-01-15,2016-12-15,2017-01-15,41.0,30.0,99.0,99.0,1.0,0.0,3
6,+++hVY1rZox/33YtvDgmKA2Frg/2qhkz12B9ylCvh8o=,2017-02-15,2017-01-15,2017-02-15,41.0,30.0,99.0,99.0,1.0,0.0,2
7,+++hVY1rZox/33YtvDgmKA2Frg/2qhkz12B9ylCvh8o=,2017-03-15,2017-02-15,2017-03-15,41.0,30.0,99.0,99.0,1.0,0.0,1
9,+++l/EXNMLTijfLBa8p2TUVVVp2aFGSuUI/h7mLmthw=,2015-02-28,2015-01-31,2015-03-19,39.0,31.0,149.0,149.0,1.0,0.0,20
10,+++l/EXNMLTijfLBa8p2TUVVVp2aFGSuUI/h7mLmthw=,2015-03-31,2015-02-28,2015-04-19,39.0,31.0,149.0,149.0,1.0,0.0,19
11,+++l/EXNMLTijfLBa8p2TUVVVp2aFGSuUI/h7mLmthw=,2015-04-30,2015-03-31,2015-05-19,39.0,31.0,149.0,149.0,1.0,0.0,18
12,+++l/EXNMLTijfLBa8p2TUVVVp2aFGSuUI/h7mLmthw=,2015-05-31,2015-04-30,2015-06-19,39.0,0.0,0.0,149.0,1.0,0.0,17
13,+++l/EXNMLTijfLBa8p2TUVVVp2aFGSuUI/h7mLmthw=,2015-06-30,2015-05-31,2015-07-19,39.0,30.0,149.0,149.0,1.0,0.0,16
